In [ ]:
!pip install --retries 0 adabelief-tf==0.2.0

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

            
import itertools as itt
import numbers
import numpy as np
import pandas as pd

from abc import abstractmethod
from typing import Iterable, Tuple, List


class BaseTimeSeriesCrossValidator:
    """
    Abstract class for time series cross-validation.
    Time series cross-validation requires each sample has a prediction time pred_time, at which the features are used to
    predict the response, and an evaluation time eval_time, at which the response is known and the error can be
    computed. Importantly, it means that unlike in standard sklearn cross-validation, the samples X, response y,
    pred_times and eval_times must all be pandas dataframe/series having the same index. It is also assumed that the
    samples are time-ordered with respect to the prediction time (i.e. pred_times is non-decreasing).
    Parameters
    ----------
    n_splits : int, default=10
        Number of folds. Must be at least 2.
    """
    def __init__(self, n_splits=10):
        if not isinstance(n_splits, numbers.Integral):
            raise ValueError(f"The number of folds must be of Integral type. {n_splits} of type {type(n_splits)}"
                             f" was passed.")
        n_splits = int(n_splits)
        if n_splits <= 1:
            raise ValueError(f"K-fold cross-validation requires at least one train/test split by setting n_splits = 2 "
                             f"or more, got n_splits = {n_splits}.")
        self.n_splits = n_splits
        self.pred_times = None
        self.eval_times = None
        self.indices = None

    @abstractmethod
    def split(self, X: pd.DataFrame, y: pd.Series = None,
              pred_times: pd.Series = None, eval_times: pd.Series = None):
        if not isinstance(X, pd.DataFrame) and not isinstance(X, pd.Series):
            raise ValueError('X should be a pandas DataFrame/Series.')
        if not isinstance(y, pd.Series) and y is not None:
            raise ValueError('y should be a pandas Series.')
        if not isinstance(pred_times, pd.Series):
            raise ValueError('pred_times should be a pandas Series.')
        if not isinstance(eval_times, pd.Series):
            raise ValueError('eval_times should be a pandas Series.')
        if y is not None and (X.index == y.index).sum() != len(y):
            raise ValueError('X and y must have the same index')
        if (X.index == pred_times.index).sum() != len(pred_times):
            raise ValueError('X and pred_times must have the same index')
        if (X.index == eval_times.index).sum() != len(eval_times):
            raise ValueError('X and eval_times must have the same index')

        self.pred_times = pred_times
        self.eval_times = eval_times
        self.indices = np.arange(X.shape[0])


class CombPurgedKFoldCV(BaseTimeSeriesCrossValidator):
    """
    Purged and embargoed combinatorial cross-validation
    As described in Advances in financial machine learning, Marcos Lopez de Prado, 2018.
    The samples are decomposed into n_splits folds containing equal numbers of samples, without shuffling. In each cross
    validation round, n_test_splits folds are used as the test set, while the other folds are used as the train set.
    There are as many rounds as n_test_splits folds among the n_splits folds.
    Each sample should be tagged with a prediction time pred_time and an evaluation time eval_time. The split is such
    that the intervals [pred_times, eval_times] associated to samples in the train and test set do not overlap. (The
    overlapping samples are dropped.) In addition, an "embargo" period is defined, giving the minimal time between an
    evaluation time in the test set and a prediction time in the training set. This is to avoid, in the presence of
    temporal correlation, a contamination of the test set by the train set.
    Parameters
    ----------
    n_splits : int, default=10
        Number of folds. Must be at least 2.
    n_test_splits : int, default=2
        Number of folds used in the test set. Must be at least 1.
    embargo_td : pd.Timedelta, default=0
        Embargo period (see explanations above).
    """
    def __init__(self, n_splits=10, n_test_splits=2, embargo_td=0):
        super().__init__(n_splits)
        if not isinstance(n_test_splits, numbers.Integral):
            raise ValueError(f"The number of test folds must be of Integral type. {n_test_splits} of type "
                             f"{type(n_test_splits)} was passed.")
        n_test_splits = int(n_test_splits)
        if n_test_splits <= 0 or n_test_splits > self.n_splits - 1:
            raise ValueError(f"K-fold cross-validation requires at least one train/test split by setting "
                             f"n_test_splits between 1 and n_splits - 1, got n_test_splits = {n_test_splits}.")
        self.n_test_splits = n_test_splits

        if embargo_td < 0:
            raise ValueError(f"The embargo time should be positive, got embargo = {embargo_td}.")
        self.embargo_td = embargo_td

    def split(self, X: pd.DataFrame, y: pd.Series = None,
              pred_times: pd.Series = None, eval_times: pd.Series = None) -> Iterable[Tuple[np.ndarray, np.ndarray]]:
        """
        Yield the indices of the train and test sets.
        Although the samples are passed in the form of a pandas dataframe, the indices returned are position indices,
        not labels.
        Parameters
        ----------
        X : pd.DataFrame, shape (n_samples, n_features), required
            Samples. Only used to extract n_samples.
        y : pd.Series, not used, inherited from _BaseKFold
        pred_times : pd.Series, shape (n_samples,), required
            Times at which predictions are made. pred_times.index has to coincide with X.index.
        eval_times : pd.Series, shape (n_samples,), required
            Times at which the response becomes available and the error can be computed. eval_times.index has to
            coincide with X.index.
        Returns
        -------
        train_indices: np.ndarray
            A numpy array containing all the indices in the train set.
        test_indices : np.ndarray
            A numpy array containing all the indices in the test set.
        """
        super().split(X, y, pred_times, eval_times)

        # Fold boundaries
        fold_bounds = [(fold[0], fold[-1] + 1) for fold in np.array_split(self.indices, self.n_splits)]
        # List of all combinations of n_test_splits folds selected to become test sets
        selected_fold_bounds = list(itt.combinations(fold_bounds, self.n_test_splits))
        
        # In order for the first round to have its whole test set at the end of the dataset
        selected_fold_bounds.reverse()

        for fold_bound_list in selected_fold_bounds:
            # Computes the bounds of the test set, and the corresponding indices
            test_fold_bounds, test_indices = self.compute_test_set(fold_bound_list)
            # Computes the train set indices
            train_indices = self.compute_train_set(test_fold_bounds, test_indices)

            yield train_indices, test_indices

    def compute_train_set(self, test_fold_bounds: List[Tuple[int, int]], test_indices: np.ndarray) -> np.ndarray:
        """
        Compute the position indices of samples in the train set.
        Parameters
        ----------
        test_fold_bounds : List of tuples of position indices
            Each tuple records the bounds of a block of indices in the test set.
        test_indices : np.ndarray
            A numpy array containing all the indices in the test set.
        Returns
        -------
        train_indices: np.ndarray
            A numpy array containing all the indices in the train set.
        """
        # As a first approximation, the train set is the complement of the test set
        train_indices = np.setdiff1d(self.indices, test_indices)
        # But we now have to purge and embargo
        for test_fold_start, test_fold_end in test_fold_bounds:
            # Purge
            train_indices = purge(self, train_indices, test_fold_start, test_fold_end)
            # Embargo
            train_indices = embargo(self, train_indices, test_indices, test_fold_end)
        return train_indices

    def compute_test_set(self, fold_bound_list: List[Tuple[int, int]]) -> Tuple[List[Tuple[int, int]], np.ndarray]:
        """
        Compute the indices of the samples in the test set.
        Parameters
        ----------
        fold_bound_list: List of tuples of position indices
            Each tuple records the bounds of the folds belonging to the test set.
        Returns
        -------
        test_fold_bounds: List of tuples of position indices
            Like fold_bound_list, but with the neighboring folds in the test set merged.
        test_indices: np.ndarray
            A numpy array containing the test indices.
        """
        test_indices = np.empty(0)
        test_fold_bounds = []
        for fold_start, fold_end in fold_bound_list:
            # Records the boundaries of the current test split
            if not test_fold_bounds or fold_start != test_fold_bounds[-1][-1]:
                test_fold_bounds.append((fold_start, fold_end))
            # If the current test split is contiguous to the previous one, simply updates the endpoint
            elif fold_start == test_fold_bounds[-1][-1]:
                test_fold_bounds[-1] = (test_fold_bounds[-1][0], fold_end)
            test_indices = np.union1d(test_indices, self.indices[fold_start:fold_end]).astype(int)
        return test_fold_bounds, test_indices


def compute_fold_bounds(cv: BaseTimeSeriesCrossValidator, split_by_time: bool) -> List[int]:
    """
    Compute a list containing the fold (left) boundaries.
    Parameters
    ----------
    cv: BaseTimeSeriesCrossValidator
        Cross-validation object for which the bounds need to be computed.
    split_by_time: bool
        If False, the folds contain an (approximately) equal number of samples. If True, the folds span identical
        time intervals.
    """
    if split_by_time:
        full_time_span = cv.pred_times.max() - cv.pred_times.min()
        fold_time_span = full_time_span / cv.n_splits
        fold_bounds_times = [cv.pred_times.iloc[0] + fold_time_span * n for n in range(cv.n_splits)]
        return cv.pred_times.searchsorted(fold_bounds_times)
    else:
        return [fold[0] for fold in np.array_split(cv.indices, cv.n_splits)]


def embargo(cv: BaseTimeSeriesCrossValidator, train_indices: np.ndarray,
            test_indices: np.ndarray, test_fold_end: int) -> np.ndarray:
    """
    Apply the embargo procedure to part of the train set.
    This amounts to dropping the train set samples whose prediction time occurs within self.embargo_dt of the test
    set sample evaluation times. This method applies the embargo only to the part of the training set immediately
    following the end of the test set determined by test_fold_end.
    Parameters
    ----------
    cv: Cross-validation class
        Needs to have the attributes cv.pred_times, cv.eval_times, cv.embargo_dt and cv.indices.
    train_indices: np.ndarray
        A numpy array containing all the indices of the samples currently included in the train set.
    test_indices : np.ndarray
        A numpy array containing all the indices of the samples in the test set.
    test_fold_end : int
        Index corresponding to the end of a test set block.
    Returns
    -------
    train_indices: np.ndarray
        The same array, with the indices subject to embargo removed.
    """
    if not hasattr(cv, 'embargo_td'):
        raise ValueError("The passed cross-validation object should have a member cv.embargo_td defining the embargo"
                         "time.")
    last_test_eval_time = cv.eval_times.iloc[cv.indices[:test_fold_end]].max()
    min_train_index = len(cv.pred_times[cv.pred_times <= last_test_eval_time + cv.embargo_td])
    if min_train_index < cv.indices.shape[0]:
        allowed_indices = np.concatenate((cv.indices[:test_fold_end], cv.indices[min_train_index:]))
        train_indices = np.intersect1d(train_indices, allowed_indices)
    return train_indices


def purge(cv: BaseTimeSeriesCrossValidator, train_indices: np.ndarray,
          test_fold_start: int, test_fold_end: int) -> np.ndarray:
    """
    Purge part of the train set.
    Given a left boundary index test_fold_start of the test set, this method removes from the train set all the
    samples whose evaluation time is posterior to the prediction time of the first test sample after the boundary.
    Parameters
    ----------
    cv: Cross-validation class
        Needs to have the attributes cv.pred_times, cv.eval_times and cv.indices.
    train_indices: np.ndarray
        A numpy array containing all the indices of the samples currently included in the train set.
    test_fold_start : int
        Index corresponding to the start of a test set block.
    test_fold_end : int
        Index corresponding to the end of the same test set block.
    Returns
    -------
    train_indices: np.ndarray
        A numpy array containing the train indices purged at test_fold_start.
    """
    time_test_fold_start = cv.pred_times.iloc[test_fold_start]
    # The train indices before the start of the test fold, purged.
    train_indices_1 = np.intersect1d(train_indices, cv.indices[cv.eval_times < time_test_fold_start])
    # The train indices after the end of the test fold.
    train_indices_2 = np.intersect1d(train_indices, cv.indices[test_fold_end:])
    

    return np.concatenate((train_indices_1, train_indices_2))

In [ ]:
import hashlib
import matplotlib.pyplot as plt

In [ ]:
TRAINING=True
PATH = '../input/lstmtry2js'

In [ ]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop=True)
    
train = train.query('weight > 0').reset_index(drop=True)

feature_tags = pd.read_csv('../input/jane-street-market-prediction/features.csv')
stock_id_features = feature_tags.query('tag_14==True')['feature']

train['stock_id'] = [hash(a.tobytes()) for a in train[['date',*stock_id_features]].values]
mintime = train['feature_64'].min()
maxtime = train['feature_64'].max()
train['feature_64'] = train['feature_64'] - mintime

In [ ]:
from sklearn.linear_model import Ridge
resp_cols = [c for c in train.columns if 'resp' in c]

c = 0.9

for resp in resp_cols:
    not_resp = [r for r in resp_cols if r != resp]
    regr = Ridge()
    regr.fit(train[not_resp].values,train[resp].values)
    
    train[f'denoised_{resp}'] = c * regr.predict(train[not_resp].values) + (1-c) * train[resp].values


In [ ]:
targets_arr = train[[c for c in train.columns if 'denoised_resp' in c]].values
targets_bool_arr = targets_arr>0

my_map = {True: 'Y', False: 'N'}
target_str_arr = np.vectorize(my_map.get)(targets_bool_arr)
target_str_arr_join = ["".join(i) for i in target_str_arr[:,:].astype(str)]

train['pattern'] = target_str_arr_join

decision = dict(zip((train.groupby('pattern').mean()['denoised_resp'] > 0).index,(train.groupby('pattern').mean()['denoised_resp'].values>0).astype(int)))
train['action'] = train['pattern'].apply(lambda x: decision[x])

In [ ]:
resp_cols = [c for c in train.columns if 'denoised_resp' in c]

In [ ]:
train['weight'] = np.clip(train['weight'],0.1,10)

In [ ]:
from collections import defaultdict
from numba import njit

def ffill_loop(arr,fill):
    mask = np.isnan(arr[0])
    arr[0][mask] = fill[mask]
    for i in range(1, len(arr)):
        mask = np.isnan(arr[i])
        arr[i][mask] = arr[i - 1][mask]
    return arr

def bfill_loop(arr,fill):
    mask = np.isnan(arr[0])
    arr[-1][mask] = fill[mask]
    for i in range(len(arr)-1, 0, -1):
        mask = np.isnan(arr[i-1])
        arr[i-1][mask] = arr[i][mask]
    return arr


In [ ]:
import gc
tf.keras.backend.clear_session()
gc.collect()

In [ ]:
import matplotlib.pyplot as plt
d = 16
time_embedding_dim = d*2
def time_embedding(time,maxtime):
    out = []
    for i in range(1,d+1):
        te = np.asarray([np.sin((time/maxtime)**(2*i/d)),np.cos((time/maxtime)**(2*i/d))])
        out.append(te)
    out = np.concatenate(out,axis=0).T
    return out
            
train[[f'feature_te_{d}' for d in range(time_embedding_dim)]] = time_embedding(train['feature_64'].values,
                                                                              train['feature_64'].values.max())
features = [c for c in train.columns if 'feature' in c and c not in stock_id_features]
features.remove('feature_64')
ffill = np.zeros(len(features))

In [ ]:
import tensorflow as tf
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, df, 
                 lookback = 10, 
                 batch_size=32, 
                 shuffle=False, 
                 classification=True, 
                 weighted=False,
                 make_stationary='no'):
        'Initialization'
        self.df = df
        self.weighted = weighted
        self.lookback = lookback
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.classification = classification
        self.make_stationary = make_stationary
        
        self.maxtime = df['feature_64'].values.max()
        
        self.indexes = np.unique(df['date'].values)
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.indexes) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        return self.__data_generation(indexes)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.unique(self.df['date'].values)
        if self.shuffle:
            random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'
        df = self.df[self.df['date'].isin(indexes)]
        
        X = []
        Y = df[resp_cols].values
        
        stock_idxs = defaultdict(list)
        
        data = df[features].values
        
        for i,s in enumerate(df['stock_id'].values):
            stock_idxs[s].append(i)
            idx = stock_idxs[s][-self.lookback:]
            sd = data[idx]
            sd = ffill_loop(sd,ffill)
            
            X.append(sd)
            
        X = tf.keras.preprocessing.sequence.pad_sequences(np.asarray(X),
                                                          maxlen=self.lookback,
                                                         value=0.0,dtype='float32')
        """
        if self.make_stationary != 'no':
            if self.make_stationary == 'diff':
                X = np.diff(X,prepend=X[0],axis=1)
            elif self.make_stationary == '1st_gradient':
                X = np.gradient(X,axis=1)
            elif self.make_stationary == '2st_gradient':
                X = np.gradient(X,2,axis=1)
            else:
                raise NotImplementedError(self.make_stationary + ' is not implemented.')
        """
        
        if self.classification:
            Y = np.where(Y>0,1,0)
        
        if self.weighted:
            return X,Y,df['weight'].values
        else:
            return X,Y

In [ ]:
batch_size=1 #days

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_addons as tfa
import tensorflow as tf

def create_model(out_dim):
    
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.LSTM(64, return_sequences=True),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.LSTM(32),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(128,activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(out_dim,activation='sigmoid')
        ]
    )
    
    try:
        from adabelief_tf import AdaBeliefOptimizer
        optimizer = AdaBeliefOptimizer(learning_rate=1e-2,
                                   beta_1=0.9,
                                   beta_2=0.999,
                                   weight_decay=1.2e-6,
                                   epsilon=1e-10)
    except:
        optimizer = 'adam'
        
    model.compile(optimizer, tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-3), 
                  metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC(curve='PR',name='auc',
                                                                                  num_thresholds=1000,
                                                                                  multi_label=True)])
    return model

model = create_model(len(resp_cols))
model.build(input_shape=(1,10,len(features)))
tf.keras.utils.plot_model(model)
model.summary()

In [ ]:
def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

In [ ]:
import random
def set_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
from tqdm import tqdm
from collections import defaultdict
lookback = 10
FOLDS = 4

th = 0.5

set_all_seeds(42)
btts = CombPurgedKFoldCV(n_splits=FOLDS, n_test_splits=2, embargo_td=5)
splits = list(btts.split(train[features],train['resp'],train['date'],train['date']))

if TRAINING:
    
    us_values = []
    th_values = []
    test_days = set()
    
    for fold, (train_idx,test_idx) in tqdm(enumerate(splits),total=len(splits)):
        train_gen = DataGenerator(train.loc[train_idx],
                                  lookback=lookback,
                                  batch_size=batch_size,
                                  classification=True,
                                 weighted=True)
        test_gen = DataGenerator(train.loc[test_idx],
                                 lookback=lookback,
                                 batch_size=batch_size,
                                 classification=True)
        
        model = create_model(len(resp_cols))
        
        model.fit(train_gen,
                  validation_data=test_gen,
                  epochs=100,
                  steps_per_epoch=len(np.unique(train.loc[train_idx]['date'].values)),
                  callbacks=[tf.keras.callbacks.EarlyStopping('val_auc',mode='max',patience=10,restore_best_weights=True),
                            tf.keras.callbacks.ReduceLROnPlateau('val_auc',mode='max',patience=3)])
        
        model.save_weights(f'model_{fold}.tf')
        
        prediction = model.predict(test_gen)
        prediction = [''.join(['Y' if p>th else 'N' for p in pred]) for pred in prediction]
        prediction = np.asarray([decision[p] if p in decision else 0.0 for p in prediction])
        
        us = utility_score_bincount(train.loc[test_idx]['date'].values,
                                   train.loc[test_idx]['weight'].values,
                                   train.loc[test_idx]['resp'].values,
                                   prediction)
        us_values.append(us)
        test_days |= set(train.loc[test_idx]['date'].values)
        
        tf.keras.backend.clear_session()
        gc.collect()
        
    print(us_values)
    print(np.mean(us_values),np.sum(us_values),np.sum(us_values)/len(test_days))
else:
    models = []
    for fold in range(len(splits)):
        model = create_model(len(resp_cols))
        model.build(input_shape=(1,10,len(features)))
        model.load_weights(f'{PATH}/model_{fold}.tf')
        model.call = tf.function(model.call, experimental_relax_shapes=True)
        models.append(model)

In [ ]:

class PredictionDataGenerator():
    def __init__(self,window_size=10):
        self.stocks = defaultdict(lambda : np.zeros((window_size,len(features)))) 
        self.date = -1
    def add_datapoint(self,df):
        date = df['date'].item()
        if date > self.date:
            self.date = date
            self.stocks.clear()
        
        key = hash(df[['date',*stock_id_features]].sum(axis=1).item())
        x = df[features[:-time_embedding_dim]].values
        t = time_embedding(x[:,64]-mintime,maxtime)
        
        x = np.concatenate([x,t],axis=1)
        self.stocks[key] = np.concatenate([self.stocks[key][1:],x],axis=0)
        self.current_stock = key
        
    def get_current_datapoint(self):
        x = self.stocks[self.current_stock]
        x = ffill_loop(x,ffill)
        x = np.expand_dims(x,0)
        return x

In [ ]:
from tqdm import tqdm
if not TRAINING:
    N = 10
    f = np.median
    import janestreet
    janestreet.competition.make_env.__called__ = False
    env = janestreet.make_env()
    
    pdg = PredictionDataGenerator(window_size=lookback)
    
    th = 0.5
    for (test_df, pred_df) in tqdm(env.iter_test()):
        pdg.add_datapoint(test_df)
        
        f0 = test_df['feature_0'].item()
        
        if test_df['weight'].item() > 0:
            x = pdg.get_current_datapoint()
            pred = np.mean([model(x,training=False).numpy() for model in models],axis=0).ravel()
            
            pred_df.action = np.where(np.median(pred) > th,1,0)
        else:
            pred_df.action = 0
        env.predict(pred_df)